## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Yellowknife,CA,2022-02-03 02:17:32,62.4560,-114.3525,-27.71,83,20,5.75,few clouds
1,1,Provideniya,RU,2022-02-03 02:17:32,64.3833,-173.3000,3.49,72,100,21.56,overcast clouds
2,2,Punta Arenas,CL,2022-02-03 02:17:33,-53.1500,-70.9167,46.51,76,75,5.75,broken clouds
3,3,Hobart,AU,2022-02-03 02:17:34,-42.8794,147.3294,63.05,78,75,1.99,broken clouds
4,4,Vaini,TO,2022-02-03 02:16:18,-21.2000,-175.2000,86.16,74,20,8.05,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)



,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
3,3,Hobart,AU,2022-02-03 02:17:34,-42.8794,147.3294,63.05,78,75,1.99,broken clouds
4,4,Vaini,TO,2022-02-03 02:16:18,-21.2000,-175.2000,86.16,74,20,8.05,few clouds
5,5,Port Elizabeth,ZA,2022-02-03 02:16:25,-33.9180,25.5701,71.10,88,75,14.97,broken clouds
7,7,Hermanus,ZA,2022-02-03 02:17:37,-34.4187,19.2345,65.89,84,0,10.74,clear sky
8,8,Avarua,CK,2022-02-03 02:17:37,-21.2078,-159.7750,84.25,74,75,17.27,broken clouds
9,9,Dunedin,NZ,2022-02-03 02:17:38,-45.8742,170.5036,59.81,90,100,3.00,light rain
11,11,San Felipe,VE,2022-02-03 02:17:39,10.3399,-68.7425,70.99,95,69,4.47,broken clouds
12,12,Codrington,AU,2022-02-03 02:17:39,-38.2667,141.9667,61.81,60,100,16.62,overcast clouds
14,14,Algodones,MX,2022-02-03 02:17:40,20.3888,-105.4773,66.74,84,0,2.46,clear sky
15,15,Rikitea,PF,2022-02-03 02:17:41,-23.1203,-134.9692,78.55,74,89,8.79,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                422
City                   422
Country                417
Date                   422
Lat                    422
Lng                    422
Max Temp               422
Humidity               422
Cloudiness             422
Wind Speed             422
Weather description    422
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
3,Hobart,AU,63.05,broken clouds,-42.8794,147.3294,
4,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,
5,Port Elizabeth,ZA,71.10,broken clouds,-33.9180,25.5701,
7,Hermanus,ZA,65.89,clear sky,-34.4187,19.2345,
8,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,
9,Dunedin,NZ,59.81,light rain,-45.8742,170.5036,
11,San Felipe,VE,70.99,broken clouds,10.3399,-68.7425,
12,Codrington,AU,61.81,overcast clouds,-38.2667,141.9667,
14,Algodones,MX,66.74,clear sky,20.3888,-105.4773,
15,Rikitea,PF,78.55,overcast clouds,-23.1203,-134.9692,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
3,Hobart,AU,63.05,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
4,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Port Elizabeth,ZA,71.10,broken clouds,-33.9180,25.5701,39 On Nile Guest House
7,Hermanus,ZA,65.89,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
8,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,Paradise Inn
9,Dunedin,NZ,59.81,light rain,-45.8742,170.5036,Scenic Hotel Southern Cross
11,San Felipe,VE,70.99,broken clouds,10.3399,-68.7425,Posada Llovizna Rodríguez
12,Codrington,AU,61.81,overcast clouds,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
14,Algodones,MX,66.74,clear sky,20.3888,-105.4773,
15,Rikitea,PF,78.55,overcast clouds,-23.1203,-134.9692,People ThankYou


In [11]:
import numpy as np

In [12]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
hotel_df.head(10)


,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
3,Hobart,AU,63.05,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
4,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Port Elizabeth,ZA,71.10,broken clouds,-33.9180,25.5701,39 On Nile Guest House
7,Hermanus,ZA,65.89,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
8,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,Paradise Inn
9,Dunedin,NZ,59.81,light rain,-45.8742,170.5036,Scenic Hotel Southern Cross
11,San Felipe,VE,70.99,broken clouds,10.3399,-68.7425,Posada Llovizna Rodríguez
12,Codrington,AU,61.81,overcast clouds,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
14,Algodones,MX,66.74,clear sky,20.3888,-105.4773,NaN
15,Rikitea,PF,78.55,overcast clouds,-23.1203,-134.9692,People ThankYou


In [17]:
# 7. Drop the rows where there is no Hotel Name
clean_hotel_df=hotel_df.dropna(axis=0, how='any')

In [18]:
# 8a. Create the output File (CSV)
output_data_file="Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country code</dt><dd>{Country}</dd>
<dt>Weather description </dt><dd>{Weather description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""



# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
 #            max_intensity=300, point_radius=4)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))